In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [33]:
df = pd.read_csv('../Datos iniciales/hechos_y_victimas.csv', encoding = 'utf-8')

Se proponen los siguientes KPI'S:


## Primer KPI:

1. Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

Para nuestro primer kpi, definimos a la tasa de homicidios en siniestros viales como el número de víctimas fatales en accidentes de tránsito por cada 100,000 habitantes en un área geográfica durante un período de tiempo específico. Su fórmula es: (Número de homicidios en siniestros viales(HSV) / Población total) * 100,000

Buscando en la web, y según la página del gobierno de Argentina ('https://www.argentina.gob.ar/caba'), en el último censo se tuvo qué la población de CABA es de 3120612 habitantes. 

In [34]:
dfk1 = df.groupby(["Año", "Semestre"])["N Victimas"].sum().reset_index()
dfk1["HSV"] = dfk1["N Victimas"]/3120612*100000
dfk1

,Año,Semestre,N Victimas,HSV
0,2016,1,67,2.147015
1,2016,2,81,2.595645
2,2017,1,81,2.595645
3,2017,2,79,2.531555
4,2018,1,76,2.435420
5,2018,2,85,2.723825
6,2019,1,59,1.890655
7,2019,2,47,1.506115
8,2020,1,31,0.993395
9,2020,2,56,1.794520


Pasamos a porcentaje:

In [54]:
dfk1["%"] = (dfk1["HSV"].pct_change() * 100).fillna(0).round(3)
dfk1

,Año,Semestre,N Victimas,HSV,%
0,2016,1,67,2.147015,0.000
1,2016,2,81,2.595645,20.896
2,2017,1,81,2.595645,0.000
3,2017,2,79,2.531555,-2.469
4,2018,1,76,2.435420,-3.797
5,2018,2,85,2.723825,11.842
6,2019,1,59,1.890655,-30.588
7,2019,2,47,1.506115,-20.339
8,2020,1,31,0.993395,-34.043
9,2020,2,56,1.794520,80.645


Encontramos dónde se cumple el kpi:

In [36]:

dfk1[dfk1["%"]<-10].reset_index()

,index,Año,Semestre,N Victimas,HSV,%
0,6,2019,1,59,1.890655,-30.588235
1,7,2019,2,47,1.506115,-20.338983
2,8,2020,1,31,0.993395,-34.042553
3,11,2021,2,42,1.345890,-23.636364


Podemos ver que el KPI1 se cumple solamente en 4 semestres, correspondientes a los años 2019, primer semestre del 2020 y segundo semestre del 2021.

In [57]:
csvkpi1 = dfk1.groupby(["Año", "Semestre"])[["N Victimas", "%"]].sum().reset_index()

csvkpi1.to_csv('../csvs/datakpi1.csv',index=False, encoding='utf-8')

## Segundo KPI:

2. Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.

Para el segunfo KPI, definimos a la cantidad de accidentes mortales de motociclistas en siniestros viales como el número absoluto de accidentes fatales en los que estuvieron involucradas víctimas que viajaban en moto en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes mortales con víctimas en moto es: (Número de accidentes mortales con víctimas en moto en el año anterior - Número de accidentes mortales con víctimas en moto en el año actual) / (Número de accidentes mortales con víctimas en moto en el año anterior) * 100

Buscamos los datos que cumplen la condicion de que el rol sea conductor y la victima sea moto:

In [41]:
KPI2 = (df["Rol"] == "CONDUCTOR") & (df["Víctima"] == "MOTO")
dfk2 = df[KPI2]
moto_year = dfk2.groupby("Año")["N Victimas"].sum().reset_index()

In [59]:
moto_year = dfk2.groupby("Año")["N Victimas"].sum().reset_index()

moto_year["%"] = ((moto_year["N Victimas"].shift(1) - moto_year["N Victimas"]) / (moto_year["N Victimas"].shift(1)) * 100).fillna(0)
moto_year

,Año,N Victimas,%
0,2016,61,0.000000
1,2017,52,14.754098
2,2018,54,-3.846154
3,2019,45,16.666667
4,2020,27,40.000000
5,2021,39,-44.444444


De antemano se observa que se cumple este KPI en 2017,2019 y 2020, en los demás años hubo incremento en las accidentes mortales para motociclistas con excepcion de 2018. En los años de Pandemia 2019-2020 se observa una disminucion de accidentes, sin embargo en 2021 se nota una tendencia considerable en el aumento de este tipo de hechos. puede deberse al retorno paulatino de la movilidad en el mundo y al incremento del uso de motocicletas como medio de transporte, derivado tambien de la pandemia (mucha gente empezó a trabajar como repatidor y similares).

In [60]:
csvkpi2 = moto_year.groupby(["Año"])[[ "%"]].sum().reset_index()

csvkpi2.to_csv('../csvs/datakpi2.csv',index=False, encoding='utf-8')

## Tercer KPI:

3. Reducir la cantidad de accidentes mortales en los Cruces en un 5% respecto al año anterior:

Definimos este KPI cómo (cantidad de victimas en cruces el año anterior - cantidad de victimas en cruces este año)/(cantidad de victimas en cruces el año anterior)*100

In [53]:
#Filtrar los accidentes en cruces
dfk3 = df[df['Cruce'] == 'SI']
dfk3 = dfk3.groupby('Año')['N Victimas'].sum().reset_index()
# Calcular el porcentaje de cambio en comparación con el año anterior
dfk3['% Cambio'] = (((dfk3['N Victimas'].shift(1) - dfk3['N Victimas']) / dfk3['N Victimas'].shift(1)) * 100).fillna(0)

dfk3

,Año,N Victimas,% Cambio
0,2016,120,0.000000
1,2017,122,-1.666667
2,2018,122,0.000000
3,2019,73,40.163934
4,2020,69,5.479452
5,2021,66,4.347826


Podemos ver cómo el KPI no se cumple en ninguno de los años, esto es preocupante y llama a las autoridades a tomar mayores medidas en estas zonas, ya sea reductores de velocidad, mayor señalizacion o semaforos.

In [62]:
csvkpi3 = dfk3.groupby(["Año"])[[ "% Cambio"]].sum().reset_index()

csvkpi3.to_csv('../csvs/datakpi3.csv',index=False, encoding='utf-8')